# MLB 85/103

In [3]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

con = sqlite3.connect("../data/CongoDB.sqlite")

## Funde

In [4]:
sql = """SELECT
           t_Obj.ObjID, 
           t_Obj.ort_kurz, 
           t_Obj.Komplex, 
           t_Obj.Individuum, 
           t_Obj.Scherbe,
           t_Obj.Anzahl,
           t_Obj.Fragm,
           t_Obj.Typ, 
           t_Obj.Gewicht,
           t_Obj.Gr_Clist, 
           t_Obj.Art 
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Ort.ort_kurz) = 'MLB')
           AND ((t_Komplex.bef_nr) = '85/103'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df

,objID,ort_kurz,Komplex,Individuum,Scherbe,Anzahl,Fragm,Typ,Gewicht,Gr_Clist,Art
0,291,MLB,85/103,1,R,1,,DAM,55,120,K
1,293,MLB,85/103,5,R,1,,BatMLB,54,120,K
2,294,MLB,85/103,7,R,1,,DON,49,120,K
3,295,MLB,85/103,11,R,1,,DAM,70,70,K
4,296,MLB,85/103,13,W,1,,Roulette,38,120,K
5,1959,MLB,85/103,3,R,1,,DON,36,70,K
6,1960,MLB,85/103,4,R,1,,DON,27,70,K
7,1961,MLB,85/103,8,R,1,,DON (?),14,70,K
8,1962,MLB,85/103,10,R,1,,MTB,52,70,K
9,1963,MLB,85/103,"12,16",R,1,3,MTB,90,120,K


In [5]:
df['Gewicht'].sum()

829

In [6]:
len(df.index)

15

In [7]:
df_pivot1 = df.pivot_table(values = 'Gewicht', index = 'Art', aggfunc = sum, fill_value = 0)
df_pivot1 = pd.DataFrame(df_pivot1)
# df_pivot1 = df_pivot1.reset_index()
df_pivot1.rename(index={'HL': 'gebrannter Lehm', 'K': 'Keramik'}, inplace=True)
df_pivot1['\%'] = (df_pivot1['Gewicht']/df_pivot1['Gewicht'].sum()*100).round(1)
df_pivot1['Gewicht'] = (df_pivot1['Gewicht'] / 1000).round(2)
df_pivot1.rename(columns={'Gewicht': 'Gewicht (kg)'}, inplace=True)

df_pivot2 = df.pivot_table(values = 'Anzahl', index = 'Art', aggfunc = sum, fill_value = 0)
df_pivot2 = pd.DataFrame(df_pivot2)
df_pivot2.rename(index={'HL': 'gebrannter Lehm', 'K': 'Keramik'}, inplace=True)
df_pivot2['\%'] = (df_pivot2['Anzahl']/df_pivot2['Anzahl'].sum()*100).round(1)

# Tabellen vereinen:
df_concat = pd.concat([df_pivot2, df_pivot1], axis = 1)
df_concat = df_concat.reset_index()
df_concat.rename(columns={'Art': 'Fundkategorie'}, inplace=True)
# df_concat.to_latex('../output/tabs/9-02_MLB85-1-3-2_Funde.tex', index = False, escape = False)
df_concat

,Fundkategorie,Anzahl,\%,Gewicht (kg),\%
0,Keramik,15,100,0.83,100


### Scherbentypen

In [5]:
df_pivot = df.pivot_table(values = 'objID', index = 'Scherbe', aggfunc = len)
df_pivot = pd.DataFrame(df_pivot)
df_pivot['Prozent'] = df_pivot['objID']/df_pivot['objID'].sum()*100
df_pivot

,objID,Prozent
Scherbe,,
R,12,80
W,3,20


### Fragmentierung

In [6]:
df_pivot = df.pivot_table(values = 'objID', index = 'Gr_Clist', aggfunc = len)
# Dummy-Werte manuell:
df_pivot['200'] = 0.
df_pivot['>200'] = 0.
df_pivot = pd.DataFrame(df_pivot)
# Prozent-Anteil
df_pivot['Prozent'] = df_pivot['objID']/df_pivot['objID'].sum()*100
df_pivot

,objID,Prozent
Gr_Clist,,
30,1,6.666667
70,7,46.666667
120,7,46.666667
200,0,0.000000
>200,0,0.000000


In [7]:
df_pivot['Prozent'].plot(kind = 'bar', color = 'gray')
plt.xlabel('Größenklassen')
plt.ylabel('Prozentanteil')

n

In [8]:
df_pivot['objID'].sum()

15.0

> **MUN 87/103** Fragmentierungsgrad

In [9]:
df['Gewicht'].plot(kind = 'hist', color = 'gray')
plt.xlabel('Gewicht [g]')
plt.ylabel('Anzahl')
plt.ylim([0,5.5]) 

(0, 5.5)

> **MUN 87/103** Histogramm des Scherbengewichts

### keramische Stilgruppen

In [10]:
df_pivot = df.pivot_table(values = 'objID', index = 'Typ', aggfunc = len)
df_pivot = pd.DataFrame(df_pivot)
df_pivot['Prozent'] = df_pivot['objID']/df_pivot['objID'].sum()*100
df_pivot = df_pivot.sort_index(by=['Prozent'], ascending=[False])
df_pivot

,objID,Prozent
Typ,,
DAM,3,20.000000
DON,3,20.000000
,2,13.333333
DON (?),2,13.333333
MTB,2,13.333333
BBL,1,6.666667
BatMLB,1,6.666667
Roulette,1,6.666667


In [11]:
fig = plt.figure(1)
fig.set_size_inches(5, 5)
df_pivot['objID'].plot(kind = 'pie', cmap = 'Paired', autopct='%.0f %%')
plt.ylabel('')

## Formen

In [12]:
sql = """SELECT
           t_Obj.ObjID, 
           t_Obj.Typ, 
           t_Obj.Scherbe,
           t_Obj.Form_Mdg, 
           t_Obj.Form_Rand,
           t_Obj.Form_Hals,
           t_Obj.Form_Schulter,
           t_Obj.Form_Bauch,
           t_Obj.Form_Boden,
           t_Obj.Form_Gef
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Ort.ort_kurz) = 'MLB')
           AND ((t_Komplex.bef_nr) = '85/103'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df

,objID,Typ,Scherbe,Form_Mdg,Form_Rand,Form_Hals,Form_Schulter,Form_Bauch,Form_Boden,Form_Gef
0,291,DAM,R,M1,R20a,,,A1,,G8d (?)
1,293,BatMLB,R,,R32alt,B1,A2,A1,,G7b
2,294,DON,R,M2/M5,B1,A1 lang,B1a,,,G6a (?)
3,295,DAM,R,M3,R21a,B3/kurz,A1,,,G3 (?)
4,296,Roulette,W,,,,A1,,,None
5,1959,DON,R,M1,B1,,A1,,,G3 (?)
6,1960,DON,R,M5,B1,,A2/leicht,,,G3 (?)
7,1961,DON (?),R,M1,R11b (?),A1,,,,
8,1962,MTB,R,M2,R13a alt,kurz,A2,,,G3 (?)
9,1963,MTB,R,M2,R18b alt,,,,,G8a (?)


### Gefäßformen

In [13]:
df_pivot = df.pivot_table(values = 'objID', index = 'Form_Gef', aggfunc = len)
df_pivot = pd.DataFrame(df_pivot)
df_pivot = df_pivot.sort_index(by=['objID'], ascending=[False])
df_pivot

,objID
Form_Gef,
G3 (?),4
,2
G8a (?),2
G8d (?),2
G4b,1
G6a (?),1
G7b,1


### Mündungsformen

In [14]:
df_pivot = df.pivot_table(values = 'objID', index = 'Form_Mdg', aggfunc = len)
df_pivot = pd.DataFrame(df_pivot)
df_pivot = df_pivot.sort_index(by=['objID'], ascending=[False])
df_pivot

,objID
Form_Mdg,
M1,5
,4
M2,2
M2/M5,1
M3,1
M4,1
M5,1


### Randformen

In [15]:
df_pivot = df.pivot_table(values = 'objID', index = 'Form_Rand', aggfunc = len)
df_pivot = pd.DataFrame(df_pivot)
df_pivot = df_pivot.sort_index(by=['objID'], ascending=[False])
df_pivot

,objID
Form_Rand,
,3
B1,3
A1,1
B3,1
C3/stark,1
R11b (?),1
R13a alt,1
R18b alt,1
R20a,1


## Verzierungen

In [16]:
sql = """SELECT
           t_Obj.objID AS objID,
           t_K_Pos.posID,
           t_K_Verz.verzID,
           [t_K_Pos].[posReihe] || ' ' || [posName] AS pos,
           t_K_Verz.verzName AS verzName,
           t_K_Verz.verzBeschr,
           t_Obj.Typ,
           t_Obj.Form_Gef
       FROM ((t_Ort LEFT JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           LEFT JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID) t_Obj INNER JOIN "t_ObjPosVerz" ON t_Obj.objID = "t_ObjPosVerz".objID
           INNER JOIN t_K_Pos ON "t_ObjPosVerz".posID = t_K_Pos.posID
           INNER JOIN t_K_Verz ON "t_ObjPosVerz".verzID = t_K_Verz.verzID
       WHERE (((t_K_Pos.posID)<>1)
           AND ((t_K_Verz.verzName) Like 'V%')
           AND ((t_Ort.ort_kurz) = 'MLB')
           AND ((t_Komplex.bef_nr) = '85/103'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
display(df.head())

,objID,t_K_Pos.posID,t_K_Verz.verzID,pos,verzName,t_K_Verz.verzBeschr,t_Obj.Typ,t_Obj.Form_Gef
0,291,2,1,1 Innen,V01a,horizontale Rille,DAM,G8d (?)
1,291,3,32,2 Rand,V09a1,Einstichreihe/diagonal,DAM,G8d (?)
2,291,4,23,3 Oberteil,V08a,vegetabilisches Roulette/Knotted String,DAM,G8d (?)
3,1962,3,75,2 Rand,V09m,gebogene Eindrücke/Kamm,MTB,G3 (?)
4,295,6,23,5 Schulter,V08a,vegetabilisches Roulette/Knotted String,DAM,G3 (?)


In [17]:
a = pd.Series({'Roulette-Verzierung':len(df.loc[df['verzName'].str.contains("V08")].index),
               'Weitere Verzierungen':len(df.loc[~df['verzName'].str.contains("V08")].index)})
a = pd.DataFrame(a)
a['Prozent'] = a[0]/a[0].sum()*100
a

,0,Prozent
Roulette-Verzierung,3,8.823529
Weitere Verzierungen,31,91.176471


In [18]:
# nur Roulette-Verzierungen herausfiltern
b = df.loc[df['verzName'].str.contains("V08")]

c = pd.Series({'vegetabilisches Roulette':len(b.loc[df['verzName'].str.contains("V08a")].index),
               'Schnitz-Roulette':len(b.loc[~df['verzName'].str.contains("V08a")].index)})
c = pd.DataFrame(c)
c['Prozent'] = c[0]/c[0].sum()*100
c

,0,Prozent
Schnitz-Roulette,1,33.333333
vegetabilisches Roulette,2,66.666667


In [19]:
df_pivot = df.pivot_table(values = 'objID', index = 'verzName', aggfunc = len)
df_pivot = pd.DataFrame(df_pivot)
# Sortierung in Aufsteigend ändern
df_pivot = df_pivot.sort_index(by = ['objID'], ascending = True)

In [20]:
fig = plt.figure()
fig.set_size_inches(5, 5)
df_pivot['objID'].plot(kind='pie', autopct='%.0f %%', cmap = 'Paired' )
plt.ylabel('')